In [1]:
import importlib
import numpy as np
import argparse
import pickle
import os
import pandas as pd

import sys
sys.path.append('/gpfs/commons/groups/gursoy_lab/ajoglekar/Projects/2023_03_01_multiwayInteractions/v0.analysis/scripts/pythonScripts/functions/')

from v1_chains import IncDFCreator, increaseIncDF_binSize
import v1_chains

import chains

In [2]:
dataDir = '/gpfs/commons/groups/gursoy_lab/ajoglekar/Projects/2023_03_01_multiwayInteractions/2023_03_01_v0_dataGathering/v0_hypergraphSimulations/getMultiwayInteractions_fromBPChains/v3.multiwayConstraints'
inputDir = f'{dataDir}/inputData/'

chainDir = 'cell_1_10000_chains_reconstructed/'

In [3]:
int1 = [[1,20,30], [5,9,15], [1,22,35,49], [5,11,20,25,40]]
int2 = [[10,20,30], [30,43,50], [10,15,23,35,43,50]]

In [4]:
## Taking from Carlos
from scipy.spatial.distance import cdist

def chain_from_file(df):
    """Given x,y,z coordinates of chains, read into array"""
    x = df[df.columns[0]] #takes first column of the df
    y = df[df.columns[1]] #similarly for the second column
    z = df[df.columns[2]] #similarly for the third column
    a = [np.array([x[i], y[i], z[i]]) for i in range(len(x))] #get all the coordinates of the monomers and store them in a list
    return a

def interactions_from_chains(chain):
    """Given an array of coordinates, convert into df and 
    find pairwise distances"""
    df = pd.DataFrame(chain)
    distances = cdist(df, df)
    return distances

In [123]:
file_name = '03009.pts'
df = pd.read_csv(os.path.join(inputDir,chainDir, file_name), sep="\s+", comment="#", header=None)

In [124]:
chain = chain_from_file(df)
distMat = interactions_from_chains(chain)

In [125]:
# importlib.reload(v1_chains)

In [137]:
creator = v1_chains.IncDFCreator(4, 120, 300, 1)

In [138]:
oneIter = creator.makeIncDF_fromChainDists_single(distMat)

In [139]:
exChain = oneIter[0]
ratios = oneIter[1]

In [140]:
numReads = exChain.shape[1]
print(numReads)
card = exChain.sum()
maxCard = card.max(0)
print(maxCard)

1127894
19.0


In [141]:
exChain.index = ["Bin"+str(i) for i in exChain.index]

In [131]:
def dfToDict(df,result_dict):
    """Takes in an incidence DF and converts to 
    a dictionary of hyperedges"""
    for col in df.columns:
        indices = df.index[df[col] == 1].tolist()
        key = '_'.join(indices)
        result_dict[key] = result_dict.get(key, 0) + 1
    return(result_dict)

inc_dict = {}
inc_dict = dfToDict(df,inc_dict)

TypeError: sequence item 0: expected str instance, int found

In [142]:
inc_dict = {}
inc_dict = chains.dfToDict(exChain,inc_dict)

iDF = chains.dictToDF(inc_dict)


In [143]:
A = ["_".join(["Bin"+str(j) for j in i]) for i in int1]

In [144]:
[x in inc_dict for x in A]

[False, True, False, False]

In [89]:
iDF.iloc[0:10,0:10]

,Read1:1,Read2:1,Read3:2,Read4:1,Read5:1,Read6:1,Read7:1,Read8:1,Read9:1,Read10:1
Bin2,1,1,0,0,0,0,0,0,0,0
Bin3,0,0,0,1,1,1,1,1,1,1
Bin4,0,0,0,0,0,0,0,0,0,0
Bin5,0,0,0,1,0,0,0,0,0,0
Bin6,0,0,0,0,1,0,0,0,0,0
Bin7,0,0,0,0,0,0,0,0,0,0
Bin8,0,0,0,0,0,0,0,0,0,0
Bin9,0,0,0,0,0,1,0,0,0,0
Bin10,1,0,1,0,0,0,1,0,0,0
Bin11,0,0,0,0,0,0,0,1,0,0
